In [22]:
import seaborn as sns
import pandas as pd


pd.set_option("display.max_columns", None)

In [23]:
df = pd.read_csv("data/1000_scenarios.csv")

df["out_of_road"].sort_values()

0         False
421917    False
421918    False
421919    False
421920    False
          ...  
551535     True
632263     True
567756     True
554647     True
604718     True
Name: out_of_road, Length: 632891, dtype: bool

In [24]:
grouped = df.groupby(by=["decision_repeat", "env_seed"])
finish = grouped.last()
finish.sort_values("cost")

Unnamed: 0  overtake_vehicle_num  velocity  \
decision_repeat env_seed                                               
5               0                314                     0  8.084753   
15              499            54200                     0  6.804013   
                500            54293                     0  7.792587   
                501            55294                     0  0.024472   
                502            55417                     0  6.790684   
...                              ...                   ...       ...   
5               786           227609                     0  1.938092   
                788           228225                     0  2.021702   
                789           228421                     0  0.107386   
10              485            73560                     0  4.758463   
15              903            97044                     0  2.008824   

                          steering  acceleration  step_energy  episode_energy  \
decision_repeat env_seed                                                        
5               0        -0.002082     -0.077891     0.035593       16.740120   
15              499       0.012021     -0.062491     0.087115        9.049153   
                500      -0.030838      0.164725     0.099421       10.332987   
                501      -1.000000     -0.133967     0.000015        6.283573   
                502       0.021279     -0.071462     0.087000        9.583163   
...                            ...           ...          ...             ...   
5               786      -0.029231     -1.000000     0.002502        5.657554   
                788      -0.028923     -1.000000     0.010634        2.646586   
                789       0.106529     -0.933045     0.000049        5.431638   
10              485       0.469093     -0.080880     0.040199        2.721157   
15              903      -0.184092      0.281838     0.023787        8.945232   

                                  policy  crash_vehicle  crash_object  \
decision_repeat env_seed                                                
5               0         EnvInputPolicy          False         False   
15              499       EnvInputPolicy          False         False   
                500       EnvInputPolicy          False         False   
                501       EnvInputPolicy          False         False   
                502       EnvInputPolicy          False         False   
...                                  ...            ...           ...   
5               786       EnvInputPolicy           True         False   
                788       EnvInputPolicy           True         False   
                789       EnvInputPolicy           True         False   
10              485       EnvInputPolicy           True         False   
15              903       EnvInputPolicy           True         False   

                          crash_building  crash_human  crash_sidewalk  \
decision_repeat env_seed                                                
5               0                  False        False           False   
15              499                False        False           False   
                500                False        False           False   
                501                False        False           False   
                502                False        False           False   
...                                  ...          ...             ...   
5               786                False        False           False   
                788                False        False           False   
                789                False        False           False   
10              485                False        False           False   
15              903                False        False           False   

                          out_of_road  arrive_dest  max_step  crash  \
decision_repeat env_seed                                        

In [28]:
grouped.cumsum().sort_values()

0           0.0
840         0.0
839         0.0
838         0.0
837         0.0
          ...  
632729    599.0
632730    599.0
632731    599.0
632718    599.0
632890    599.0
Name: cost, Length: 632891, dtype: float64